In [10]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
import mimetypes
import base64
import csv
# If modifying these scopes, delete the file token.pickle.
#SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
SCOPES = ['https://www.googleapis.com/auth/gmail.send']

In [16]:
def create_message(sender, to, replyto, subject, message_text):
  """Create a message for an email.

  Args:
    sender: Email address of the sender.
    to: Email address of the receiver.
    subject: The subject of the email message.
    message_text: The text of the email message.

  Returns:
    An object containing a base64url encoded email object.
  """
  message = MIMEText(message_text, 'html')
  message['to'] = to
  message['from'] = sender
  message['subject'] = subject
  message['reply-to'] = replyto
    
  b64_bytes = base64.urlsafe_b64encode(message.as_bytes())
  b64_string = b64_bytes.decode()

  #return {'raw': base64.urlsafe_b64encode(message.as_string())}
  return {'raw': b64_string}

In [17]:
def send_message(service, user_id, message):
  """Send an email message.

  Args:
    service: Authorized Gmail API service instance.
    user_id: User's email address. The special value "me"
    can be used to indicate the authenticated user.
    message: Message to be sent.

  Returns:
    Sent Message.
  """
  try:
    message = (service.users().messages().send(userId=user_id, body=message)
               .execute())
    print("Message Id: %s" % message['id'])
    return message
  except Exception as error:
    print('An error occurred: %s' % error)

In [18]:
def send():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API
    #results = service.users().labels().list(userId='me').execute()
    #labels = results.get('labels', [])

    #if not labels:
    #    print('No labels found.')
    #else:
    #    print('Labels:')
    #    for label in labels:
    #        print(label['name'])
    send_message(service, 'me', message)

In [39]:
with open("testing_list.csv", "r", encoding="utf8") as f:
    reader = csv.reader(f, delimiter=",")
    for line in enumerate(reader):
        #print('line[{}] = {}'.format(i, line))
        #print(line[0].strip(","))
        userid = line[1][0].zfill(4)
        #fn = "certs\\KUAR_certificate__Data Set "+userid+".pdf"
        name = line[1][1]
        email = line[1][2]
        
        print(userid, name, email)
        
        sender = "kuar@ku.edu.tr"
        replyto = 'no-reply@ku.edu.tr' 
        subject = 'KUAR Designlab Bilgilendirme'
        #subject = subject.encode("latin1")
        #subject = unicode(subject, 'latin1')
        
        message_text = """<p>-------- L&uuml;tfen bu e-postayı cevaplamayınız.&nbsp; &nbsp;--------</p>
        <p>Merhaba """ + name +""",</p>
        <p>Aramıza hoşgeldin! Bundan sonraki s&uuml;re&ccedil;le&nbsp;ilgili sizi bilgilendirmek istiyoruz:</p>
        <ol>
        <li>2021 yılı i&ccedil;erisinde herhangi bir zamanda sizin <strong>profilinize uygun</strong> &ccedil;alışmalarımız olduğunda sizinle temas kuruyor olacağız.</li>
        <li>Size<strong> hemen </strong>bir &ccedil;alışma g&ouml;ndermemizi l&uuml;tfen beklemeyiniz.&nbsp; &nbsp;</li>
        <li>Bu e-mail hesabının <strong>spam olarak işaretlenmediğine</strong>&nbsp;l&uuml;tfen dikkat ediniz.</li>
        <li>Size g&ouml;nderilen g&ouml;revlerin <strong>10 tanesini </strong>tamamladığınızda <strong>Deneyimli Katılımcı Sertifikası</strong> alacağınızı hatırlatmak isteriz.</li>
        </ol>
        <p>Sağlıklı g&uuml;nler dileriz.</p>
        <p>KUARteam</p>
        <p><a href="http://kuar.ku.edu.tr/" target="_blank" rel="noopener">Ko&ccedil; University&nbsp;&ndash; Ar&ccedil;elik Research Center for Creative Industries</a></p>"""
        
        message = create_message(sender, email, replyto, subject, message_text)
        
        #send()
        

0000 No Reply no-reply@ku.edu.tr
Message Id: 1768efef4e559181


In [7]:
#send()